Para la KPI2 necesitaremos los archivos, a modo de prueba: fhvhv_tripdata_2024-01, fhv_tripdata_2024-01, taxi+_zone_lookup y taxi_zones.dbf.

Para calcular la tasa de cambio en el número de usuarios de servicios de taxis.

**Columnas**:
| pickup_datetime | dropOff_datetime | dispatching | PUlocationID | DOlocationID | passenger_count |

* Agrupar los datos por número de pasajeros trasladados.
* Calcular la tasa de cambio en el número de usuarios trasladados en taxi.

* La tasa de cambio en el número de usuarios del servicio se calculará utilizando la siguiente fórmula:

* Tasa de Cambio = (NúmeroUsuariosFinalPeríodo − NúmeroUsuariosComienzoPeríodo) / NúmeroUsuariosComienzoPeríodo × 100


Abriremos archivos de FHVHV y FHV, además de las datas de zona.

In [24]:
# Librerías:
# Librerías
import pandas as pd
import numpy as np

import pyarrow.parquet as pq 
import os
from pandas import DataFrame
from dbfread import DBF

import seaborn as sns
import matplotlib as mpl
import plotly.express as px

import matplotlib.pyplot as plt

# para eliminar warning en python
import warnings
warnings.simplefilter('ignore')


Abriendo datos de FHVHV:

In [25]:
# Assumiendo que el archivo está en la carpeta
# Cambiar al directorio con el archivo
file_path = "C:/Users/56988/Desktop/NYdrive/fhvhv_tripdata_2024-01.parquet"
os.chdir("C:/Users/56988/Desktop/NYdrive/")

In [26]:
# Dataframe Taxis FHVHV
df_fhvhv_ene24 = pd.read_parquet(file_path)

In [27]:
df_fhvhv_ene24.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [28]:
df_fhvhv_ene24.head(2)

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2024-01-01 00:21:47,2024-01-01 00:25:06,2024-01-01 00:28:08,2024-01-01 01:05:39,161,158,2.83,...,4.05,2.75,0.0,0.0,40.18,N,N,N,N,N
1,HV0003,B03404,B03404,2024-01-01 00:10:56,2024-01-01 00:11:08,2024-01-01 00:12:53,2024-01-01 00:20:05,137,79,1.57,...,0.89,2.75,0.0,0.0,6.12,N,N,N,N,N


Abrir archivo FHV:

In [29]:
# Abrir data de FHV
file_path = "C:/Users/56988/Desktop/NYdrive/fhv_tripdata_2024-01.parquet"
os.chdir("C:/Users/56988/Desktop/NYdrive/")

In [30]:
df_fhv_ene24 = pd.read_parquet(file_path)

In [31]:
df_fhv_ene24.head(2)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00053,2024-01-01 00:15:00,2024-01-01 02:13:00,NaN,NaN,NaN,B00014
1,B00111,2024-01-01 00:30:00,2024-01-01 02:37:00,NaN,NaN,NaN,B00111


In [32]:
df_fhv_ene24.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

Abrir data de los taxis+zones, para asociarlos por zona y borough

In [33]:
# Abre el archivo DBF
dbf = DBF("DATASETORIGEN/taxi_zones.dbf")

# Convierte los datos a un DataFrame
df = DataFrame(iter(dbf))

In [34]:
df.columns

Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID',
       'borough'],
      dtype='object')

Unión de dataframes:

In [35]:
# datos de taxi unidos con la data de zona de taxi

# Unir los DataFrames
df_unido_fhvhvene24 = pd.merge(df_fhvhv_ene24, df, left_on="PULocationID", right_on="LocationID")

In [36]:
df_unido_fhvhvene24.head(2)

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough
0,HV0003,B03404,B03404,2024-01-01 00:21:47,2024-01-01 00:25:06,2024-01-01 00:28:08,2024-01-01 01:05:39,161,158,2.83,...,N,N,N,N,161,0.035804,0.000072,Midtown Center,161,Manhattan
1,HV0003,B03404,B03404,2024-01-01 00:10:56,2024-01-01 00:11:08,2024-01-01 00:12:53,2024-01-01 00:20:05,137,79,1.57,...,N,N,N,N,137,0.046108,0.000116,Kips Bay,137,Manhattan


In [37]:
# Ver columnas (nombres)
df_unido_fhvhvene24.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag', 'OBJECTID', 'Shape_Leng',
       'Shape_Area', 'zone', 'LocationID', 'borough'],
      dtype='object')

In [38]:
# Cantidad de valores nulos en cada columna
null_counts_fhvhvene24= df_unido_fhvhvene24.isnull().sum()

In [39]:
null_counts_fhvhvene24

hvfhs_license_num             0
dispatching_base_num          0
originating_base_num    5232431
request_datetime              0
on_scene_datetime       5232431
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_miles                    0
trip_time                     0
base_passenger_fare           0
tolls                         0
bcf                           0
sales_tax                     0
congestion_surcharge          0
airport_fee                   0
tips                          0
driver_pay                    0
shared_request_flag           0
shared_match_flag             0
access_a_ride_flag            0
wav_request_flag              0
wav_match_flag                0
OBJECTID                      0
Shape_Leng                    0
Shape_Area                    0
zone                          0
LocationID                    0
borough                       0
dtype: int64

*Se aplicó una revisión de duplicados, y no apareció ninguno, no se observa el código porque son 20 millones de datos.

In [40]:
# Agrupar por las columnas deseadas, primero al recoger al pasajero:
grouped_data_ini = df_unido_fhvhvene24.groupby(["pickup_datetime", "PULocationID", 
                            "trip_miles", "trip_time", "zone", "borough"])

In [41]:
# Sumar las millas de viaje
sum_trip_miles_ini = grouped_data_ini["trip_miles"].sum().reset_index(name="total_trip_miles")

In [42]:
sum_trip_miles_ini

,pickup_datetime,PULocationID,trip_miles,trip_time,zone,borough,total_trip_miles
0,2024-01-01 00:00:00,21,2.260,649,Bensonhurst East,Brooklyn,2.260
1,2024-01-01 00:00:00,36,2.714,1054,Bushwick North,Brooklyn,2.714
2,2024-01-01 00:00:00,69,1.710,732,East Concourse/Concourse Village,Bronx,1.710
3,2024-01-01 00:00:00,138,8.177,795,LaGuardia Airport,Queens,8.177
4,2024-01-01 00:00:00,144,1.040,380,Little Italy/NoLiTa,Manhattan,1.040
...,...,...,...,...,...,...,...
19658826,2024-01-31 23:59:58,64,11.490,1180,Douglaston,Queens,11.490
19658827,2024-01-31 23:59:58,236,12.260,1700,Upper East Side North,Manhattan,12.260
19658828,2024-01-31 23:59:59,17,1.094,332,Bedford,Brooklyn,1.094
19658829,2024-01-31 23:59:59,163,23.390,2411,Midtown North,Manhattan,23.390


In [43]:
# Crea dos nuevas columnas para la fecha y la hora
sum_trip_miles_ini.loc[:, "fecha_pic"] = sum_trip_miles_ini["pickup_datetime"].dt.date
sum_trip_miles_ini.loc[:, "hora_pic"] = sum_trip_miles_ini["pickup_datetime"].dt.time

In [44]:
# DataFrame con año, mes y día separados
sum_trip_miles_ini.loc[:, "agno_pic"] = sum_trip_miles_ini["pickup_datetime"].dt.year
sum_trip_miles_ini.loc[:, "mes_pic"] = sum_trip_miles_ini["pickup_datetime"].dt.month
sum_trip_miles_ini.loc[:, "dia_pic"] = sum_trip_miles_ini["pickup_datetime"].dt.day

In [45]:
# Crea una nueva columna 'hora_exacta' extrayendo la hora de la columna 'hora'
sum_trip_miles_ini.loc[:, "hora_exacta_pic"] = sum_trip_miles_ini["hora_pic"].apply(lambda x: x.hour)

In [46]:
sum_trip_miles_ini.head(2)

,pickup_datetime,PULocationID,trip_miles,trip_time,zone,borough,total_trip_miles,fecha_pic,hora_pic,agno_pic,mes_pic,dia_pic,hora_exacta_pic
0,2024-01-01,21,2.260,649,Bensonhurst East,Brooklyn,2.260,2024-01-01,00:00:00,2024,1,1,0
1,2024-01-01,36,2.714,1054,Bushwick North,Brooklyn,2.714,2024-01-01,00:00:00,2024,1,1,0


In [47]:
# Agrupado por "dia_pic" y calculado el total hasta el día 15.
grouped_df_15_count = sum_trip_miles_ini[sum_trip_miles_ini["dia_pic"] <= 15].groupby("dia_pic")["dia_pic"].agg(dia_pic_count=("sum"))

In [48]:
grouped_df_15_count

,dia_pic_count
dia_pic,
1,638196
2,956238
3,1494612
4,2289856
5,3282290
6,4253064
7,3798277
8,4155464
9,5617530


In [49]:
# Agrupado por "dia_pic" y calculado el total del 16 al 31.
grouped_df_31_count = sum_trip_miles_ini[sum_trip_miles_ini["dia_pic"] >= 16].groupby("dia_pic")["dia_pic"].agg(dia_pic_count=("sum"))

In [50]:
grouped_df_31_count

,dia_pic_count
dia_pic,
16,9862512
17,11552826
18,12308130
19,14030151
20,16488480
21,14231427
22,12559008
23,13532901
24,14917968


Tasa de cambio de la recogida de pasajeros FHVHV (se compararon los primeros 15 días con los 15 posteriores):

In [51]:
# Entonces si queremos saber la tasa de cambio en el número de usuarios de los servicios de taxis:
# Realizamos el cálculo de números de usuarios servicio taxi:
# (NumeroUsuariosFinalPeriodo - NumeroUsuariosComienzoPeriodo) / NumeroUsuariosComienzoPeriodo * 100

Tasa_cambio_Usuarios_taxis = ((18886564 - 8386455) / 18886564 * 100)
# Formateo con dos decimales
tasa_cambio_formateada = round(Tasa_cambio_Usuarios_taxis, 2)

print(f"Tasa de cambio de usuarios de taxis: {tasa_cambio_formateada}%")

Tasa de cambio de usuarios de taxis: 55.6%


* Para realizar la tasa de cambio, se compararon los primeros 15 días con los posteriores 15 días de Enero 2024, para lo que es recogida de pasajeros FHVHV
(FHV de alto volumen, vehículos de alquiler).
* Son más de 19 658 831 datos.
* Si se esperaba un aumento del 10%, en este caso se cumple (período muy corto de tiempo), aumenta en un 55.6% el número de usuarios.

In [52]:
# Promediar las millas de viaje
avg_trip_time_ini = grouped_data_ini["trip_time"].mean().reset_index(name="prom_trip_miles")
avg_trip_time_ini

,pickup_datetime,PULocationID,trip_miles,trip_time,zone,borough,prom_trip_miles
0,2024-01-01 00:00:00,21,2.260,649,Bensonhurst East,Brooklyn,649.0
1,2024-01-01 00:00:00,36,2.714,1054,Bushwick North,Brooklyn,1054.0
2,2024-01-01 00:00:00,69,1.710,732,East Concourse/Concourse Village,Bronx,732.0
3,2024-01-01 00:00:00,138,8.177,795,LaGuardia Airport,Queens,795.0
4,2024-01-01 00:00:00,144,1.040,380,Little Italy/NoLiTa,Manhattan,380.0
...,...,...,...,...,...,...,...
19658826,2024-01-31 23:59:58,64,11.490,1180,Douglaston,Queens,1180.0
19658827,2024-01-31 23:59:58,236,12.260,1700,Upper East Side North,Manhattan,1700.0
19658828,2024-01-31 23:59:59,17,1.094,332,Bedford,Brooklyn,332.0
19658829,2024-01-31 23:59:59,163,23.390,2411,Midtown North,Manhattan,2411.0


In [53]:
# Agrupar por las columnas deseadas, ahora al dejar al pasajero:
grouped_data_lleg = df_unido_fhvhvene24.groupby(["dropoff_datetime", "DOLocationID", 
                            "trip_miles", "trip_time", "zone", "borough"])

In [54]:
# Count the number of trips for each DOLocationID
grouped_data_lleg_trips = grouped_data_lleg.size()

In [55]:
grouped_data_lleg_trips

dropoff_datetime     DOLocationID  trip_miles  trip_time  zone                      borough  
2024-01-01 00:02:35  80            0.575       141        Bushwick North            Brooklyn     1
2024-01-01 00:02:37  183           0.000       121        Pelham Bay                Bronx        1
2024-01-01 00:02:42  74            0.533       137        East Harlem North         Manhattan    1
2024-01-01 00:03:11  107           0.430       168        Gramercy                  Manhattan    1
2024-01-01 00:03:38  119           0.627       214        Washington Heights North  Manhattan    1
                                                                                                ..
2024-02-01 01:38:27  246           19.182      7985       JFK Airport               Queens       1
2024-02-01 01:48:42  265           65.510      7690       TriBeCa/Civic Center      Manhattan    1
2024-02-01 02:00:41  265           217.820     14141      JFK Airport               Queens       1
2024-02-01 02:0

In [56]:
# Sumar las millas de viaje
sum_trip_miles_lleg = grouped_data_lleg["trip_miles"].sum().reset_index(name="total_trip_miles")

In [57]:
sum_trip_miles_lleg

,dropoff_datetime,DOLocationID,trip_miles,trip_time,zone,borough,total_trip_miles
0,2024-01-01 00:02:35,80,0.575,141,Bushwick North,Brooklyn,0.575
1,2024-01-01 00:02:37,183,0.000,121,Pelham Bay,Bronx,0.000
2,2024-01-01 00:02:42,74,0.533,137,East Harlem North,Manhattan,0.533
3,2024-01-01 00:03:11,107,0.430,168,Gramercy,Manhattan,0.430
4,2024-01-01 00:03:38,119,0.627,214,Washington Heights North,Manhattan,0.627
...,...,...,...,...,...,...,...
19658826,2024-02-01 01:38:27,246,19.182,7985,JFK Airport,Queens,19.182
19658827,2024-02-01 01:48:42,265,65.510,7690,TriBeCa/Civic Center,Manhattan,65.510
19658828,2024-02-01 02:00:41,265,217.820,14141,JFK Airport,Queens,217.820
19658829,2024-02-01 02:03:16,265,183.200,11746,Flushing,Queens,183.200


*trip_time está en segundos y miles en millas

In [58]:
# Crea dos nuevas columnas para la fecha y la hora
sum_trip_miles_lleg.loc[:, "fecha_off"] = sum_trip_miles_lleg["dropoff_datetime"].dt.date
sum_trip_miles_lleg.loc[:, "hora_off"] = sum_trip_miles_lleg["dropoff_datetime"].dt.time

In [59]:
# DataFrame con año, mes y día separados
sum_trip_miles_lleg.loc[:, "agno_off"] = sum_trip_miles_lleg["dropoff_datetime"].dt.year
sum_trip_miles_lleg.loc[:, "mes_off"] = sum_trip_miles_lleg["dropoff_datetime"].dt.month
sum_trip_miles_lleg.loc[:, "dia_off"] = sum_trip_miles_lleg["dropoff_datetime"].dt.day

In [60]:
# Crea una nueva columna 'hora_exacta' extrayendo la hora de la columna 'hora'
sum_trip_miles_lleg.loc[:, "hora_exacta_off"] = sum_trip_miles_lleg["hora_off"].apply(lambda x: x.hour)

In [61]:
sum_trip_miles_lleg.head(2)

,dropoff_datetime,DOLocationID,trip_miles,trip_time,zone,borough,total_trip_miles,fecha_off,hora_off,agno_off,mes_off,dia_off,hora_exacta_off
0,2024-01-01 00:02:35,80,0.575,141,Bushwick North,Brooklyn,0.575,2024-01-01,00:02:35,2024,1,1,0
1,2024-01-01 00:02:37,183,0.000,121,Pelham Bay,Bronx,0.000,2024-01-01,00:02:37,2024,1,1,0


In [62]:
# Agrupado por "dia_off" y calculado el total hasta el día 15.
grouped_df_15_count_off = sum_trip_miles_lleg[sum_trip_miles_lleg["dia_off"] <= 15].groupby("dia_off")["dia_off"].agg(dia_off_count=("sum"))

In [63]:
grouped_df_15_count_off

,dia_off_count
dia_off,
1,640336
2,956100
3,1492803
4,2285152
5,3264655
6,4251036
7,3837099
8,4156208
9,5617188


In [64]:
# Agrupado por "dia_off" y calculado el total desde el día 16 al 31.
grouped_df_31_count_off = sum_trip_miles_lleg[sum_trip_miles_lleg["dia_off"] >= 16].groupby("dia_off")["dia_off"].agg(dia_off_count=("sum"))

In [65]:
grouped_df_31_count_off

,dia_off_count
dia_off,
16,9867616
17,11534721
18,12279114
19,13957913
20,16428140
21,14428554
22,12576564
23,13509234
24,14893152


Tasa de cambio de dejada de pasajeros FHVHV:

In [66]:
# Entonces si queremos saber la tasa de cambio en el número de usuarios de los servicios de taxis:
# Realizamos el cálculo de números de usuarios servicio taxi:
# (NumeroUsuariosFinalPeriodo - NumeroUsuariosComienzoPeriodo) / NumeroUsuariosComienzoPeriodo * 100

Tasa_cambio_Usuarios_taxis = ((18844063 - 8422065) / 8422065 * 100)
# Formateo con dos decimales
tasa_cambio_formateada = round(Tasa_cambio_Usuarios_taxis, 2)

print(f"Tasa de cambio de usuarios de taxis: {tasa_cambio_formateada}%")

Tasa de cambio de usuarios de taxis: 123.75%


Este aumento significativo, puede deberse al viaje compartido, lo cual hace aumentar la ruta y el tiempo dejando pasajeros.

In [67]:
# Promediar las millas de viaje
avg_trip_time_lleg = grouped_data_lleg["trip_time"].mean().reset_index(name="prom_trip_miles")
avg_trip_time_lleg

,dropoff_datetime,DOLocationID,trip_miles,trip_time,zone,borough,prom_trip_miles
0,2024-01-01 00:02:35,80,0.575,141,Bushwick North,Brooklyn,141.0
1,2024-01-01 00:02:37,183,0.000,121,Pelham Bay,Bronx,121.0
2,2024-01-01 00:02:42,74,0.533,137,East Harlem North,Manhattan,137.0
3,2024-01-01 00:03:11,107,0.430,168,Gramercy,Manhattan,168.0
4,2024-01-01 00:03:38,119,0.627,214,Washington Heights North,Manhattan,214.0
...,...,...,...,...,...,...,...
19658826,2024-02-01 01:38:27,246,19.182,7985,JFK Airport,Queens,7985.0
19658827,2024-02-01 01:48:42,265,65.510,7690,TriBeCa/Civic Center,Manhattan,7690.0
19658828,2024-02-01 02:00:41,265,217.820,14141,JFK Airport,Queens,14141.0
19658829,2024-02-01 02:03:16,265,183.200,11746,Flushing,Queens,11746.0


Se pueden observar las millas totales de viaje de inicio y de llegada del usuario de taxis FHVH (FHV de alto volumen, viajes de alquiler) en Enero del 2024.

Para viajes FHV:

In [68]:
# datos de taxi unidos con la data de zona de taxi

# Unir los DataFrames
df_unido_fhvene24 = pd.merge(df_fhv_ene24, df, left_on="PUlocationID", right_on="LocationID")

In [69]:
df_unido_fhvene24.tail(2)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough
279512,B03320,2024-01-31 23:58:00,2024-02-01 00:22:00,138.0,13.0,NaN,B03320,138,0.107467,0.000537,LaGuardia Airport,138,Queens
279513,B03320,2024-01-31 23:01:00,2024-02-01 00:17:00,107.0,265.0,NaN,B03404,107,0.038041,0.000075,Gramercy,107,Manhattan


In [70]:
# Crea dos nuevas columnas para la fecha y la hora de recogida de pasajeros
df_unido_fhvene24.loc[:, "fecha_pic2"] = df_unido_fhvene24["pickup_datetime"].dt.date
df_unido_fhvene24.loc[:, "hora_pic2"] = df_unido_fhvene24["pickup_datetime"].dt.time

In [71]:
# DataFrame con año, mes y día separados
df_unido_fhvene24.loc[:, "agno_pic2"] = df_unido_fhvene24["pickup_datetime"].dt.year
df_unido_fhvene24.loc[:, "mes_pic2"] = df_unido_fhvene24["pickup_datetime"].dt.month
df_unido_fhvene24.loc[:, "dia_pic2"] = df_unido_fhvene24["pickup_datetime"].dt.day

In [72]:
# Crea una nueva columna 'hora_exacta' extrayendo la hora de la columna 'hora'
df_unido_fhvene24.loc[:, "hora_exacta_pic2"] = df_unido_fhvene24["hora_pic2"].apply(lambda x: x.hour)

In [73]:
# Crea dos nuevas columnas para la fecha y la hora de repartir pasajeros
df_unido_fhvene24.loc[:, "fecha_off2"] = df_unido_fhvene24["dropOff_datetime"].dt.date
df_unido_fhvene24.loc[:, "hora_off2"] = df_unido_fhvene24["dropOff_datetime"].dt.time

In [74]:
# DataFrame con año, mes y día separados
df_unido_fhvene24.loc[:, "agno_off2"] = df_unido_fhvene24["dropOff_datetime"].dt.year
df_unido_fhvene24.loc[:, "mes_off2"] = df_unido_fhvene24["dropOff_datetime"].dt.month
df_unido_fhvene24.loc[:, "dia_off2"] = df_unido_fhvene24["dropOff_datetime"].dt.day

In [75]:
# Crea una nueva columna 'hora_exacta' extrayendo la hora de la columna 'hora'
df_unido_fhvene24.loc[:, "hora_exacta_off2"] = df_unido_fhvene24["hora_off2"].apply(lambda x: x.hour)

In [76]:
df_unido_fhvene24.head(2)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,OBJECTID,Shape_Leng,Shape_Area,...,agno_pic2,mes_pic2,dia_pic2,hora_exacta_pic2,fecha_off2,hora_off2,agno_off2,mes_off2,dia_off2,hora_exacta_off2
0,B00171,2024-01-01 00:52:00,2024-01-01 00:57:00,151.0,75.0,NaN,B00256,151,0.054890,0.000129,...,2024,1,1,0,2024-01-01,00:57:00,2024,1,1,0
1,B00254,2024-01-01 00:30:55,2024-01-01 00:46:58,161.0,239.0,NaN,B02970,161,0.035804,0.000072,...,2024,1,1,0,2024-01-01,00:46:58,2024,1,1,0


In [77]:
# Estos no son viajes compartidos!!! No hay valores para SR_Flag, no son compartidos. Es decir, 279 514 dentro de 20 millones, no son viajes compartidos.
df_unido_fhvene24["SR_Flag"].isnull().sum()

279514

In [78]:
# Cantidad de valores nulos en cada columna
null_counts_fhvene24= df_unido_fhvene24.isnull().sum()

In [79]:
null_counts_fhvene24

dispatching_base_num           0
pickup_datetime                0
dropOff_datetime               0
PUlocationID                   0
DOlocationID               11610
SR_Flag                   279514
Affiliated_base_number         0
OBJECTID                       0
Shape_Leng                     0
Shape_Area                     0
zone                           0
LocationID                     0
borough                        0
fecha_pic2                     0
hora_pic2                      0
agno_pic2                      0
mes_pic2                       0
dia_pic2                       0
hora_exacta_pic2               0
fecha_off2                     0
hora_off2                      0
agno_off2                      0
mes_off2                       0
dia_off2                       0
hora_exacta_off2               0
dtype: int64

In [80]:
# Seleccionar las columnas para nueva DF:
selected_df = df_unido_fhvene24[["PUlocationID", "DOlocationID", "zone", "borough", "dia_pic2", "dia_off2"]]

In [81]:
selected_df # TRATAR DE PASAR EL TIPO DE DATO A NUMÉRICO O ENTERO EN ESTA PARTE   dasboard para el viernes

,PUlocationID,DOlocationID,zone,borough,dia_pic2,dia_off2
0,151.0,75.0,Manhattan Valley,Manhattan,1,1
1,161.0,239.0,Midtown Center,Manhattan,1,1
2,140.0,233.0,Lenox Hill East,Manhattan,1,1
3,261.0,239.0,World Trade Center,Manhattan,1,1
4,262.0,229.0,Yorkville East,Manhattan,1,1
...,...,...,...,...,...,...
279509,140.0,76.0,Lenox Hill East,Manhattan,31,1
279510,39.0,97.0,Canarsie,Brooklyn,31,1
279511,82.0,82.0,Elmhurst,Queens,31,31
279512,138.0,13.0,LaGuardia Airport,Queens,31,1


In [85]:
# Convertir columnas a int
cols_to_convert = ['PUlocationID', 'DOlocationID', 'dia_pic2', 'dia_off2']
selected_df[cols_to_convert] = selected_df[cols_to_convert].fillna(0).astype(int)

In [86]:
# Group by the specified columns
grouped_df = selected_df.groupby(['PUlocationID', 'zone', 'borough', 'dia_pic2'])

# Sum 'dia_pic2' within each group
grouped_df_sum = grouped_df['dia_pic2'].sum()

In [87]:
grouped_df_sum

PUlocationID  zone            borough    dia_pic2
1             Newark Airport  EWR        1            26
                                         2            58
                                         3            90
                                         4            80
                                         5            85
                                                    ... 
263           Yorkville West  Manhattan  27          216
                                         28          168
                                         29          261
                                         30          390
                                         31          434
Name: dia_pic2, Length: 7559, dtype: int32

In [94]:
filtered_df = grouped_df_sum[grouped_df_sum.index.get_level_values('dia_pic2') <= 15]
total_dia_pic2_15 = filtered_df.sum()

In [95]:
total_dia_pic2_15   

989158

In [102]:
filtered_df = grouped_df_sum[grouped_df_sum.index.get_level_values('dia_pic2') >= 16]
total_dia_pic2_31 = filtered_df.sum()

In [103]:
total_dia_pic2_31

3696155

Tasa de cambio de la recogida de pasajeros FHV:

In [104]:
# Entonces si queremos saber la tasa de cambio en el número de usuarios de los servicios de taxis:
# Realizamos el cálculo de números de usuarios servicio taxi:
# (NumeroUsuariosFinalPeriodo - NumeroUsuariosComienzoPeriodo) / NumeroUsuariosComienzoPeriodo * 100

Tasa_cambio_Usuarios_taxis = ((3696155 - 989158) / 989158 * 100)
# Formateo con dos decimales
tasa_cambio_formateada = round(Tasa_cambio_Usuarios_taxis, 2)

print(f"Tasa de cambio de usuarios de taxis: {tasa_cambio_formateada}%")

Tasa de cambio de usuarios de taxis: 273.67%


In [105]:
# Group by the specified columns
grouped_df_off2 = selected_df.groupby(['DOlocationID', 'zone', 'borough', 'dia_off2'])

# Sum 'dia_pic2' within each group
grouped_df_sum_off2 = grouped_df_off2['dia_off2'].sum()

In [106]:
grouped_df_sum_off2

DOlocationID  zone            borough    dia_off2
0             Baisley Park    Queens     2            2
                                         17          17
                                         23          46
              Bath Beach      Brooklyn   1            2
                                         4            4
                                                     ..
265           Yorkville West  Manhattan  17          68
                                         18          18
                                         19          38
                                         26          26
                                         29          29
Name: dia_off2, Length: 135211, dtype: int32

In [107]:
filtered_df = grouped_df_sum_off2[grouped_df_sum_off2.index.get_level_values('dia_off2') <= 15]
total_dia_off2_15 = filtered_df.sum()

In [109]:
total_dia_off2_15

991343

In [108]:
filtered_df_off2 = grouped_df_sum_off2[grouped_df_sum_off2.index.get_level_values('dia_off2') >= 16]
total_dia_off2_31 = filtered_df_off2.sum()

In [110]:
total_dia_off2_31

3691250

Tasa de cambio de la dejada de pasajeros FHV:

In [111]:
# Entonces si queremos saber la tasa de cambio en el número de usuarios de los servicios de taxis:
# Realizamos el cálculo de números de usuarios servicio taxi:
# (NumeroUsuariosFinalPeriodo - NumeroUsuariosComienzoPeriodo) / NumeroUsuariosComienzoPeriodo * 100

Tasa_cambio_Usuarios_taxis = ((3691250 - 991343) / 991343 * 100)
# Formateo con dos decimales
tasa_cambio_formateada = round(Tasa_cambio_Usuarios_taxis, 2)

print(f"Tasa de cambio de usuarios de taxis: {tasa_cambio_formateada}%")

Tasa de cambio de usuarios de taxis: 272.35%


*Los viajes de dejar pasajeros, FHV son los viajes de alquiler, fuera de los taxis amarillos y verdes, 
la tasa de cambio descendió un poco (1,32%), respecto de la recogida, se puede decir que no hay grandes 
variaciones en los viajes o la dejada de pasajeros, ya que la mayoría no era viaje compartido.